# Prep Data for Figures
In a few cases, we need to create reduced data files to build the figures in our paper.

In [39]:
import os
import sys

import numpy as np
import h5py
import matplotlib
import matplotlib.pyplot as plt
import seaborn
import astropy.units as u
from astropy.coordinates import SkyCoord
from scipy.interpolate import splev
from fiasco import list_elements
import distributed
import plasmapy.atomic

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.analysis import AIATimelags, DistributedAIACube
from synthesizAR.atomic import EmissionModel

sys.path.append('../paper/python/')
from formatting import get_figsize,hist_step,heating_palette,qualitative_palette

import warnings
warnings.filterwarnings('ignore',category=UserWarning,)

%matplotlib inline

## Heating
Save the hydrodynamic quantities for a single sample loop to avoid having to track the entire loop parameters datasets (many GBs). Additionally, we'll save the heating rate because we want to plot this later on.

In [14]:
heating = ['high_frequency', 'intermediate_frequency', 'low_frequency']

In [15]:
i_loop=680

In [18]:
for h in heating:
    with h5py.File(f'/storage-home/w/wtb2/data/timelag_synthesis_v2/{h}/loop_parameters.h5', 'r') as hf:
        with h5py.File(f'../paper/data/{h}/loop_parameters.h5', 'w') as hf2:
            hf.copy(f'loop{i_loop:06d}', hf2)
            q = np.loadtxt(f'/storage-home/w/wtb2/data/timelag_synthesis_v2/{h}/hydro_results/loop{i_loop:06d}')[:,-1]
            ds = hf2[f'loop{i_loop:06d}'].create_dataset('heating_rate', data=q)
            ds.attrs['unit'] = 'erg cm^-3 s^-1'

## Effective Response Functions
Save the effective AIA response functions for the elements that we are concerned with in addition to the elemental components of each response.

In [25]:
aia = InstrumentSDOAIA([0,1]*u.s,observer_coordinate=None)

In [29]:
em = EmissionModel.restore('/storage-home/w/wtb2/data/timelag_synthesis_v2/base_emission_model.json')

In [32]:
with h5py.File('../paper/data/effective_aia_response.h5', 'w') as h5:
    for channel in aia.channels:
        grp = h5.create_group(f"{channel['name']}")
        counts = np.zeros(em.temperature.shape+em.density.shape)
        components = {}                     
        for ion in em:
            wvl,emiss = em.get_emissivity(ion)
            if wvl is None or emiss is None:
                continue
            response = splev(wvl.value, channel['wavelength_response_spline'])
            response = np.where(response < 0., 0., response)
            tmp = np.dot(emiss.value, response)
            tmp *= ion.abundance.value*ion.ioneq.value[:,np.newaxis]/em.density.value/4./np.pi
            counts += tmp
            if ion.element_name in components:
                components[ion.element_name] += tmp 
            else:
                components[ion.element_name] = tmp
        grp.create_dataset('response', data=counts)
        for k in components:
            grp.create_dataset(k, data=components[k])

## Timelag Example
Now, we'll save the pixel-averaged intensities and corresponding timelags for a sample region of the cooling case. This is just to show an example of the timelag calculation in 1D.

In [2]:
channels = [94,131,171,193,211,335]
channel_pairs = [(94,335),
                 (94,171),
                 (94,193),
                 (94,131),
                 (94,211),
                 (335,131),
                 (335,193),
                 (335,211), 
                 (335,171),
                 (211,131),
                 (211,171),
                 (211,193),
                 (193,171),
                 (193,131),
                 (171,131),]

In [3]:
cluster = distributed.LocalCluster(n_workers=32,threads_per_worker=2,)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34439 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 64 Memory: 270.38 GB


In [4]:
cooling = AIATimelags(*[DistributedAIACube.from_files(
    f'/storage-home/w/wtb2/data/timelag_synthesis_v2/cooling/nei/SDO_AIA/{c}/*.fits') for c in channels])

In [5]:
m = cooling[0].maps[0]
bc = SkyCoord(-315*u.arcsec,-335*u.arcsec,frame=m.coordinate_frame)
tc = SkyCoord(-310*u.arcsec,-330*u.arcsec,frame=m.coordinate_frame)

In [20]:
with h5py.File('../paper/data/cooling/timelag_1d_example.h5', 'w') as h5:
    #timeseries
    grp = h5.create_group('timeseries')
    for c in channels[:]:
        ds = grp.create_dataset(f'{c}',data=cooling.make_timeseries(c, (bc.Tx,bc.Ty), (tc.Tx,tc.Ty),))
        ds.attrs['unit'] = cooling[cooling.channels.index(c)].maps[0].meta['bunit']
    ds = grp.create_dataset('time', data=cooling[0].time.value)
    ds.attrs['unit'] = cooling[0].time.unit.to_string()
    # Cross-correlations
    grp = h5.create_group('cross_correlations')
    for ca,cb in channel_pairs:
        ds = grp.create_dataset(f'{ca}_{cb}', data=cooling.correlation_1d(ca, cb, (bc.Tx,bc.Ty), (tc.Tx,tc.Ty)))
        ds.attrs['unit'] = ''
    ds = grp.create_dataset('timelags', data=cooling.timelags.value)
    ds.attrs['unit'] = cooling.timelags.unit.to_string()